# Text Normalization

## Character Level

### Unicode

유니코드는 1990년대 초 문자코드 문제를 근본적으로 해결해 보고자 제안된 국제 표준코드이다. 현재 The Unicode Consortium에서 이 프로젝트를 계속 진행 중이다. 유니코드에서 한자와 관련된 코드는 크게 한중일 통합한자(CJK Unified Ideographs), 한중일 호환한자(CJK Compatibility Ideographs), 한중일 부수(CJK Radicals/KangXi Radicals)로 구성되어 있다. 

* CJK Unified Ideographs (4E00~9FCF, 20944종): 일반적으로 한자를 나타내는 문자 코드
  - CJK Extension A (3400~4DBF, 6592종)
  - CJK Extension B (20000~2A6DF, 42720종)
  - CJK Extension C (2A700~2B73F, 4160종)
  - CJK Extension D (2B740~2B81F, 224종)
* CJK Compatibility Ideographs (F900~FAFF, 512종) : 모양이 같고 음가가 다른 문자 코드 
  - CJK Compatibility Ideographs Supplement (2F800~2FA1F, 544종)
* CJK Radicals/KangXi Radicals (2F00~2FDF, 224종): 부수와 관련된 문자 코드
  - CJK Radicals Supplement (2E80~2EFF, 128종)
  - CJK Strokes (31C0~31EF, 48종)
  - Ideographic Description Characters (2FF0~2FFF, 16종)
  

Unicode는 글자를 컴퓨터가 이해할 수 있는 고유 번호로 표시한 것이다. 유니코드의 경우 16진수 4자리 (또는 5자리)로 구성되어 있다. 

In [1]:
c_by = "補" # byte string
c_uni = u'補' # unicode string

print( c_by, type( c_by ) )
print( c_uni, type( c_uni ) )
print( c_by.encode('utf-8') )
print( c_by.encode('utf-8').decode('utf-8') )

補 <class 'str'>
補 <class 'str'>
b'\xe8\xa3\x9c'
補


In [2]:
c_int = ord( c_uni )
new_chr1 = chr( c_int )

print( "Character to Integer:", c_int )
print( "Integer to Character:", new_chr1 )

Character to Integer: 35036
Integer to Character: 補


In [4]:
c_unicode = '{:04x}'.format( c_int )
c_int2 = int( c_unicode, 16 )
new_chr2 = chr( c_int2 )

print( "Integer to Unicode:", c_unicode )
print( "Unicode to Integer:", c_int2 )
print( "Integer to Character:", new_chr2 )

Integer to Unicode: 88dc
Unicode to Integer: 35036
Integer to Character: 補


In [6]:
import unicodedata

print( "unicode category:", unicodedata.category( c_uni ) )
print( "unicode name:", unicodedata.name( c_uni ) )
# Get numeric value of second character


unicode category: Lo
unicode name: CJK UNIFIED IDEOGRAPH-88DC


### Duplicates(다중코드자)

유니코드는 형태가 같은 글자를 기준으로 구성 되었는데, 구축 과정에서 음가가 다른 한자가 중복 생성되었다. 이 글자들은 한중일 호환한자(CJK Compatibility Ideographs)에 담겨 있으며, 적합한 통합한자로 맵핑되어 있다. 그 예는 다음과 같다.  

```
不(U+F967) → 不(U+4E0D)
更(U+F901) → 更(U+66F4)
里(U+F9E9) → 里(U+91CC)
六(U+F9D1) → 六(U+516D)
```

다중코드자는 육안으로 보았을 때 동일한 형태를 가지지만, 내부적으로는 서로 다른 code를 가지고 있기 때문에 다른 글자로 인식된다. 따라서 이를 보다 정확한 결과를 얻고자 한다면 텍스트 분석 전에 이를 먼저 통합해 주어야 한다. 


In [8]:
from collections import Counter

chrs = "不不"
print( Counter( list(chrs)) )
print( ['{:04x}'.format( ord(c) ) for c in chrs ] )


Counter({'不': 1, '不': 1})
['f967', '4e0d']


In [9]:
chrs_merged = chrs.replace( '不', '不' )
print( Counter( list( chrs_merged )) )
print( ['{:04x}'.format( ord(c) ) for c in chrs_merged ] )


Counter({'不': 2})
['4e0d', '4e0d']


### Variants(이체자)

한자에는 다른 형태의 글자가 동일한 의미를 가질 때가 있다. 특히 한중일 한자의 경우, 같은 글자를 다른 형태로 사용하기도 한다. 이런 경우 역시 분석 전에 통합해 준다면 더 좋은 결과를 얻을 수 있다. 

```
尚(U+5C1A) → 尙(U+5C19)
為(U+70BA) → 爲(U+7232)
垒(U+5792) → 壘(U+58D8)
胆(U+80C6) → 膽(U+81BD)
```

다중코드자의 경우 기계적인 통합이 가능하지만, 이체자의 경우 신중을 기해야 할 때가 있다. 현대 중국에서는 後를 后로 사용하고 있지만, 后의 경우 황후라는 의미로 사용되는 경우도 있다. 이러한 예는 雲-云, 髮-發-发 등 적지 않다. 따라서 이를 한쪽으로 통합할 경우 의미의 왜곡을 피할 수 없게 된다. 



## Term Level

이를 극복하기 위해서는 단어 수준의 통합이 대안이 될 수 있다. 发을 髮이나 發과 기계적으로 통합할 수 없지만, 毛发은 毛髮과 发汗은 發汗과 통합할 수 있기 때문이다. 이를 위해서는 글자가 아닌 단어 수준의 매칭 테이블이 필요하다. 

단어 수준의 통합은 글자 수준에서 통합할 수 없었던 단어들을 통합할 수 있다는 장점도 가지고 있다. 예를 들어 한의학 고문헌에서 인삼은 人蔘, 人參, 仁參 등으로 다양하게 표기된다. 이를 간단히 나타내면 `[人仁][蔘參參𠫰]`와 같다. 인삼에 대해 최소 8가지 표기법이 존재하는 셈이다. 이러한 예는 셀 수 없을 만큼 많다. 

그러나 단어 수준의 매칭 테이블을 만드는데는 많은 노력이 소요된다. 대안으로는 word2vec 등의 word embedding 기법을 이용하는 것이다. word2vec은 의미가 유사한 단어를 찾는데 유용하게 사용될 수 있다. 따라서 대량의 텍스트를 bi-gram 기준으로 word2vec으로 임베딩 한 뒤에 유사한 단어쌍을 추출해 내고, 그 결과를 검토하는 방법을 취한다면 단어 수준의 매칭 테이블을 더 간단하게 구축할 수 있을 것이다. 

## Refs

Unicode
* [Unicode 12.0 Character Code Charts](http://www.unicode.org/charts/)
* [python / Unicode HOWTO](https://docs.python.org/3.7/howto/unicode.html)
* [pythonsheets/Unicode](https://www.pythonsheets.com/notes/python-unicode.html)
* [unicode-table](https://unicode-table.com/kr/#control-character)

References
* [Chinese Text Project/Dictionary](https://ctext.org/dictionary.pl?if=en)

Apps
* [Hanzi Text Normalizer](http://info.mediclassics.kr/apps/hanzitn/index.html)

Librarys
* [Open Chinese Convert](https://github.com/BYVoid/OpenCC)
* [HanziJS](https://github.com/nieldlr/Hanzi)

Articles
*  [이정현. 유니코드 한자 검색의 문제점 및 개선방안. 정보화정책, 2012:19.](https://nia.or.kr/common/board/Download.do?bcIdx=9560&cbIdx=65684&fileNo=1)
